In [1]:
today_date = '2026-01-05'

StatementMeta(, 3e0e31d5-5d6e-48d5-bd2f-6e61a97e3900, 3, Finished, Available, Finished)

In [2]:
abfs_path = 'abfss://Dev_Workspace@onelake.dfs.fabric.microsoft.com/Lakehouse_sales.Lakehouse/Files/landing'

partition_path = f"/Processing_date={today_date}"

complete_path = abfs_path + partition_path
display(complete_path)

StatementMeta(, 3e0e31d5-5d6e-48d5-bd2f-6e61a97e3900, 4, Finished, Available, Finished)

'abfss://Dev_Workspace@onelake.dfs.fabric.microsoft.com/Lakehouse_sales.Lakehouse/Files/landing/Processing_date=2026-01-05'

In [4]:
from pyspark.sql.types import StructType , StructField,StringType,IntegerType,DoubleType,ArrayType,DateType

#Define the schema

v_schema = StructType ([
    StructField("Row_ID",  StringType() ,True),
    StructField("Order_ID",  StringType() ,True) ,
    StructField("Order_Date" , DateType() ,True),
    StructField("Ship_Date" , DateType(),True) ,
    StructField("Ship_Mode" , StringType() ,True),
    StructField("Customer_ID"  ,StringType() ,True),
    StructField("Customer_Name" , StringType() ,True),
    StructField("Segment" , StringType() ,True),
    StructField("Postal_Code" , StringType() ,True),
    StructField("City" , StringType() ,True),
    StructField("State" , StringType() ,True),
    StructField("Country" , StringType() ,True),
    StructField("Region" , StringType() ,True),
    StructField("Market" , StringType() ,True),
    StructField("Product_ID"  ,StringType() ,True),
    StructField("Category"  ,StringType() ,True),
    StructField("Sub_Category"  ,StringType() ,True),
    StructField("Product_Name"  ,StringType() ,True),
    StructField("Sales"  ,DoubleType() ,True),
    StructField("Quantity"  ,IntegerType() ,True),
    StructField("Discount" , DoubleType() ,True),
    StructField("Profit"  ,DoubleType() ,True),
    StructField("Shipping_Cost" , DoubleType() ,True),
    StructField("Order_Priority"  ,StringType() ,True),
    StructField("Month" , StringType() ,True),
    StructField("Year" , StringType() ,True)
    ])

StatementMeta(, 3e0e31d5-5d6e-48d5-bd2f-6e61a97e3900, 6, Finished, Available, Finished)

In [5]:
df = spark.read.format('csv').option('header','True').schema(v_schema).load(complete_path)
display(df)

StatementMeta(, 3e0e31d5-5d6e-48d5-bd2f-6e61a97e3900, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ae1e873c-6417-4674-a239-e9bd34fac373)

In [10]:
df.createOrReplaceTempView('t_new_data')

StatementMeta(, 3e0e31d5-5d6e-48d5-bd2f-6e61a97e3900, 12, Finished, Available, Finished)

In [11]:
%%sql
select * from t_new_data

StatementMeta(, 3e0e31d5-5d6e-48d5-bd2f-6e61a97e3900, 13, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 26 fields>

In [14]:
Fabric_tblsales_bronze = 'abfss://Dev_Workspace@onelake.dfs.fabric.microsoft.com/Lakehouse_sales.Lakehouse/Tables/dbo/tblsales_bronze'
try:
    spark.read.format('delta').load(Fabric_tblsales_bronze).createOrReplaceTempView('t_tblsales_bronze')
except:
        v_create_table = f"""CREATE TABLE IF NOT EXISTS tblsales_bronze (
        Row_ID  string ,
        Order_ID  string ,
        Order_Date  date ,
        Ship_Date  date ,
        Ship_Mode  string ,
        Customer_ID  string ,
        Customer_Name  string ,
        Segment  string ,
        Postal_Code  string ,
        City  string ,
        State  string ,
        Country  string ,
        Region  string ,
        Market  string ,
        Product_ID  string ,
        Category  string ,
        Sub_Category  string ,
        Product_Name  string ,
        Sales  DOUBLE ,
        Quantity  int ,
        Discount  DOUBLE ,
        Profit  DOUBLE ,
        Shipping_Cost  DOUBLE ,
        Order_Priority  string ,
        Month  string ,
        Year  string ,
        processing_date date
        )"""

        spark.sql(v_create_table)
        spark.read.format('delta').load(Fabric_tblsales_bronze).createOrReplaceTempView('t_tblsales_bronze')


StatementMeta(, 3e0e31d5-5d6e-48d5-bd2f-6e61a97e3900, 16, Finished, Available, Finished)

In [15]:
%%sql
select * from t_tblsales_bronze

StatementMeta(, 3e0e31d5-5d6e-48d5-bd2f-6e61a97e3900, 17, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 27 fields>

In [16]:
sql_statement = f"""MERGE INTO tblsales_bronze as target
					USING t_new_data as source
					on target.Order_ID = source.Order_ID and target.Customer_ID = source.Customer_ID

					WHEN MATCHED THEN
						UPDATE SET 
						target.Row_ID = source.Row_ID,
						target.Order_ID  = source.Order_ID,
						target.Order_Date = source.Order_Date,
						target.Ship_Date  = source.Ship_Date,
						target.Ship_Mode  =  source.Ship_Mode,
						target.Customer_ID = source.Customer_ID,
						target.Customer_Name  = source.Customer_Name,
						target.Segment  = source.Segment,
						target.Postal_Code  = source.Postal_Code ,
						target.City  = source.City,
						target.State  = source.State ,
						target.Country  = source.Country ,
						target.Region  = source.Region,
						target.Market  = source.Market ,
						target.Product_ID  = source.Product_ID,
						target.Category  = source.Category,
						target.Sub_Category  = source.Sub_Category ,
						target.Product_Name  = source.Product_Name,
						target.Sales  = source.Sales,
						target.Quantity  = source.Quantity ,
						target.Discount  = source.Discount,
						target.Profit  = source.Profit,
						target.Shipping_Cost  = source.Shipping_Cost,
						target.Order_Priority  = source.Order_Priority,
						target.Month  = source.Month,
						target.Year  = source.Year ,
						target.processing_date = '{today_date}'

					WHEN NOT MATCHED THEN
						INSERT (Row_ID,
								Order_ID,
								Order_Date,
								Ship_Date,
								Ship_Mode,
								Customer_ID,
								Customer_Name,
								Segment,
								Postal_Code,
								City,
								State,
								Country,
								Region,
								Market,
								Product_ID,
								Category,
								Sub_Category,
								Product_Name,
								Sales,
								Quantity,
								Discount,
								Profit,
								Shipping_Cost,
								Order_Priority,
								Month,
								Year,
								processing_date)
								VALUES
								(
								source.Row_ID,
								source.Order_ID,
								source.Order_Date,
								source.Ship_Date,
								source.Ship_Mode,
								source.Customer_ID,
								source.Customer_Name,
								source.Segment,
								source.Postal_Code,
								source.City,
								source.State,
								source.Country,
								source.Region,
								source.Market,
								source.Product_ID,
								source.Category,
								source.Sub_Category,
								source.Product_Name,
								source.Sales,
								source.Quantity,
								source.Discount,
								source.Profit,
								source.Shipping_Cost,
								source.Order_Priority,
								source.Month,
								source.Year,
								'{today_date}'
								)"""
spark.sql(sql_statement).show()

StatementMeta(, 3e0e31d5-5d6e-48d5-bd2f-6e61a97e3900, 18, Finished, Available, Finished)

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|             2018|               0|               0|             2018|
+-----------------+----------------+----------------+-----------------+



In [17]:
%%sql
select * from tblsales_bronze

StatementMeta(, 3e0e31d5-5d6e-48d5-bd2f-6e61a97e3900, 19, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 27 fields>